<a href="https://colab.research.google.com/github/volkfox/CS221-Final_Project/blob/master/RPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AWS notebook v2

This colab sheet is an RPS training playground. 
For it to work, runtime has to be supported by GPU with suffiicent memory (does not happen often on free tier)

In [ ]:
# @kernel-restart
# 

input = """She sat at the window of the train, her head thrown back, one leg stretched across to the empty seat
before her. The window frame trembled with the speed of the motion, the pane hung over empty
darkness, and dots of light slashed across the glass as luminous streaks, once in a while.

Her leg, sculptured by the tight sheen of the stocking, its long line running straight, over an arched
instep, to the tip of a foot in a high-heeled pump, had a feminine elegance that seemed out of place in
the dusty train car and oddly incongruous with the rest of her. She wore a battered camel's hair coat
that had been expensive, wrapped shapelessly about her slender, nervous body. The coat collar was
raised to the slanting brim of her hat. A sweep of brown hair fell back, almost touching the line of her
shoulders. Her face was made of angular planes, the shape of her mouth clear-cut, a sensual mouth
held closed with inflexible precision. She kept her hands in the coat pockets, her posture taut, as if
she resented immobility, and unfeminine, as if she were unconscious of her own body and that it was
a woman's body. She sat listening to the music. It was a symphony of triumph. The notes flowed up,
they spoke of rising and they were the rising itself, they were the essence and the form of upward
motion, they seemed to embody every human act and thought that had ascent as its motive. It was a
sunburst of sound, breaking out of hiding and spreading open. It had the freedom of release and the
tension of purpose. It swept space clean, and left nothing but the joy of an unobstructed effort. Only a
faint echo within the sounds spoke of that from which the music had escaped, but spoke in laughing
astonishment at the discovery that there was no ugliness or pain, and there never had had to be. It was
the song of an immense deliverance.

She thought: For just a few moments — while this lasts — it is all right to surrender completely — to
forget everything and just permit yourself to feel. She thought: Let go — drop the controls — this is it.
Somewhere on the edge of her mind, under the music, she heard the sound of train wheels. They
knocked in an even rhythm, every fourth knock accented, as if stressing a conscious purpose. She
could relax, because she heard the wheels. She listened to the symphony, thinking: This is why the
wheels have to be kept going, and this is where they're going.

She had never heard that symphony before, but she knew that it was written by Richard Halley. She
recognized the violence and the magnificent intensity. She recognized the style of the theme; it was a
clear, complex melody — at a time when no one wrote melody any longer. . . . She sat looking up at the
ceiling of the car, but she did not see it and she had forgotten where she was. She did not know
whether she was hearing a frill symphony orchestra or only the theme; perhaps she was hearing the
orchestration in her own mind.

She thought dimly that there had been premonitory echoes of this theme in all of Richard Halley's
work, through all the years of his long struggle, to the day, in his middle-age, when fame struck him
suddenly and knocked him out. This — she thought, listening to the symphony — had been the goal of
his struggle. She remembered half-hinted attempts in his music, phrases that promised it, broken bits
of melody that started but never quite reached it; when Richard Halley wrote this, he . . . She sat up
straight. When did Richard Halley write this?"""


In [14]:
# @kernel-restart
# 

input = """She sat at the window of the train, her head thrown back, one leg stretched across to the empty seat
before her. The window frame trembled with the speed of the motion, the pane hung over empty
darkness, and dots of light slashed across the glass as luminous streaks, once in a while.

Her leg, sculptured by the tight sheen of the stocking, its long line running straight, over an arched
instep, to the tip of a foot in a high-heeled pump, had a feminine elegance that seemed out of place in
the dusty train car and oddly incongruous with the rest of her. She wore a battered camel's hair coat
that had been expensive, wrapped shapelessly about her slender, nervous body. The coat collar was
raised to the slanting brim of her hat. A sweep of brown hair fell back, almost touching the line of her
shoulders. Her face was made of angular planes, the shape of her mouth clear-cut, a sensual mouth
held closed with inflexible precision. She kept her hands in the coat pockets, her posture taut, as if
she resented immobility, and unfeminine, as if she were unconscious of her own body and that it was
a woman's body. She sat listening to the music. It was a symphony of triumph. The notes flowed up,
they spoke of rising and they were the rising itself, they were the essence and the form of upward
motion, they seemed to embody every human act and thought that had ascent as its motive. It was a
sunburst of sound, breaking out of hiding and spreading open. It had the freedom of release and the
tension of purpose. It swept space clean, and left nothing but the joy of an unobstructed effort. Only a
faint echo within the sounds spoke of that from which the music had escaped, but spoke in laughing
astonishment at the discovery that there was no ugliness or pain, and there never had had to be. It was
the song of an immense deliverance."""


In [ ]:
# does not work on AWS!

# upload your content file once
from google.colab import files
uploaded=files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# one-time
# install dependencies if needed 
!pip install sacremoses
!pip install gpt-2-simple 
!pip install nltk
!pip install tensorflow-gpu==1.14
!pip install tensorflow-hub 

In [3]:
# @kernel restart
import sys, getopt
import gc
from pathlib import Path
from sacremoses import MosesDetokenizer
import gpt_2_simple as gpt2
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

/home/ubuntu/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [ ]:
# one-time
# download one-time tokenizer file
nltk.download('punkt')

In [4]:
# @kernel restart
# GPT-2 released three models
model_name="117M"
#model_name="124M"
#model_name="345M"

# this is your style for GPT-2 finetuning
style = "Nabokov-All.txt"
inputfile = 'Rand'
# content input and output
outputfile = inputfile+"_"+model_name+"_"+style
statsfile = outputfile+".stats"

# hyperparameters
temperature = 1.0
minsimilarity = 0.68
# lexeme bounds
minwords = 4
maxwords = 15
seedLexemes = 5
nsamples = 10000

bestLexemes = []
output = ""

# get around resource exhaustion for embeddings
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

In [5]:
# @kernel restart
# init tokenizer and sentence embedder
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
dt = MosesDetokenizer()

module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"
embed = hub.Module(module_url)
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# download GPT-2 model (only needed once)
gpt2.download_gpt2(model_name=model_name)


In [6]:
# run always!
# set number of steps for finetuning or load the pretrained model
# if a model is already mounted, no training required

sess = gpt2.start_tf_sess()
# load an existing, fine-tuned model
# gpt2.load_gpt2(sess)
#
# finetuning frequently fails in collab based on resource exhaustion
# Good training duration results in a loss of around 0.1-0.2 (over 10K iterations)
# GPT-2 can generate coherent output even on low finetuning, but with a loss of style

# gpt2.finetune(sess, style, model_name=model_name, overwrite=True, steps=1000)
gpt2.finetune(sess, style, model_name=model_name, restore_from = 'latest', overwrite=True, steps=10000)

# fine-tune from scratch
#gpt2.finetune(sess, style, model_name=model_name, overwrite=True, steps=1000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Loading checkpoint checkpoint/run1/model-2000
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000


...

[11485 | 42954.75] loss=0.17 avg=0.21
[11486 | 42959.11] loss=0.21 avg=0.21
[11487 | 42963.45] loss=0.17 avg=0.21
[11488 | 42967.79] loss=0.25 avg=0.21
[11489 | 42972.13] loss=0.18 avg=0.21
[11490 | 42976.48] loss=0.23 avg=0.21
[11491 | 42980.82] loss=0.16 avg=0.21
[11492 | 42985.18] loss=0.21 avg=0.21
[11493 | 42989.53] loss=0.20 avg=0.21
[11494 | 42993.87] loss=0.22 avg=0.21
[11495 | 42998.22] loss=0.23 avg=0.21
[11496 | 43002.57] loss=0.20 avg=0.21
[11497 | 43006.91] loss=0.16 avg=0.21
[11498 | 43011.26] loss=0.17 avg=0.21
[11499 | 43015.61] loss=0.29 avg=0.21
[11500 | 43019.96] loss=0.22 avg=0.21
======== SAMPLE 1 ========
 rage the lights of the street) as the two girls walked hand in hand across the threshold (“touchées and shadows à la montagne,” “nostrils and shadows,” to quote Matthew Arnold’s lovely motto), he exclaimed with awe that such a young girl should remain in her past—as it was—even in this present. Controlling himself, he bounced glances at the girl who felt his gaz

[11601 | 43492.00] loss=0.19 avg=0.21
[11602 | 43496.36] loss=0.20 avg=0.21
[11603 | 43500.71] loss=0.18 avg=0.21
[11604 | 43505.10] loss=0.24 avg=0.21
[11605 | 43509.48] loss=0.27 avg=0.21
[11606 | 43513.85] loss=0.20 avg=0.21
[11607 | 43518.22] loss=0.19 avg=0.21
[11608 | 43522.61] loss=0.21 avg=0.21
[11609 | 43526.99] loss=0.22 avg=0.21
[11610 | 43531.36] loss=0.18 avg=0.21
[11611 | 43535.70] loss=0.16 avg=0.21
[11612 | 43540.08] loss=0.25 avg=0.21
[11613 | 43544.46] loss=0.30 avg=0.21
[11614 | 43548.86] loss=0.18 avg=0.21
[11615 | 43553.25] loss=0.23 avg=0.21
[11616 | 43557.64] loss=0.19 avg=0.21
[11617 | 43562.03] loss=0.25 avg=0.21
[11618 | 43566.42] loss=0.26 avg=0.21
[11619 | 43570.78] loss=0.18 avg=0.21
[11620 | 43575.17] loss=0.25 avg=0.21
[11621 | 43579.56] loss=0.22 avg=0.21
[11622 | 43583.95] loss=0.19 avg=0.21
[11623 | 43588.34] loss=0.20 avg=0.21
[11624 | 43592.72] loss=0.20 avg=0.21
[11625 | 43597.08] loss=0.25 avg=0.21
[11626 | 43601.44] loss=0.25 avg=0.21
[11627 | 436

[11705 | 43963.86] loss=0.20 avg=0.21
[11706 | 43968.23] loss=0.17 avg=0.21
[11707 | 43972.59] loss=0.21 avg=0.21
[11708 | 43976.98] loss=0.19 avg=0.21
[11709 | 43981.35] loss=0.19 avg=0.21
[11710 | 43985.72] loss=0.27 avg=0.21
[11711 | 43990.08] loss=0.16 avg=0.21
[11712 | 43994.46] loss=0.20 avg=0.21
[11713 | 43998.84] loss=0.21 avg=0.21
[11714 | 44003.21] loss=0.27 avg=0.21
[11715 | 44007.58] loss=0.27 avg=0.21
[11716 | 44011.97] loss=0.28 avg=0.21
[11717 | 44016.36] loss=0.19 avg=0.21
[11718 | 44020.73] loss=0.24 avg=0.21
[11719 | 44025.11] loss=0.19 avg=0.21
[11720 | 44029.51] loss=0.24 avg=0.21
[11721 | 44033.88] loss=0.18 avg=0.21
[11722 | 44038.26] loss=0.21 avg=0.21
[11723 | 44042.63] loss=0.23 avg=0.21
[11724 | 44047.01] loss=0.20 avg=0.21
[11725 | 44051.38] loss=0.22 avg=0.21
[11726 | 44055.76] loss=0.22 avg=0.21
[11727 | 44060.16] loss=0.20 avg=0.21
[11728 | 44064.54] loss=0.23 avg=0.21
[11729 | 44068.92] loss=0.25 avg=0.21
[11730 | 44073.32] loss=0.25 avg=0.21
[11731 | 440

[11829 | 44523.54] loss=0.16 avg=0.21
[11830 | 44527.92] loss=0.18 avg=0.20
[11831 | 44532.30] loss=0.33 avg=0.21
[11832 | 44536.70] loss=0.21 avg=0.21
[11833 | 44541.11] loss=0.21 avg=0.21
[11834 | 44545.49] loss=0.20 avg=0.21
[11835 | 44549.86] loss=0.17 avg=0.21
[11836 | 44554.23] loss=0.24 avg=0.21
[11837 | 44558.62] loss=0.21 avg=0.21
[11838 | 44563.00] loss=0.24 avg=0.21
[11839 | 44567.37] loss=0.20 avg=0.21
[11840 | 44571.76] loss=0.22 avg=0.21
[11841 | 44576.14] loss=0.28 avg=0.21
[11842 | 44580.52] loss=0.25 avg=0.21
[11843 | 44584.91] loss=0.18 avg=0.21
[11844 | 44589.28] loss=0.19 avg=0.21
[11845 | 44593.65] loss=0.18 avg=0.21
[11846 | 44598.03] loss=0.17 avg=0.21
[11847 | 44602.40] loss=0.14 avg=0.21
[11848 | 44606.77] loss=0.19 avg=0.21
[11849 | 44611.14] loss=0.30 avg=0.21
[11850 | 44615.52] loss=0.17 avg=0.21
[11851 | 44619.91] loss=0.21 avg=0.21
[11852 | 44624.30] loss=0.15 avg=0.21
[11853 | 44628.69] loss=0.20 avg=0.21
[11854 | 44633.10] loss=0.25 avg=0.21
[11855 | 446

[11939 | 45021.80] loss=0.22 avg=0.21
[11940 | 45026.19] loss=0.21 avg=0.21
[11941 | 45030.57] loss=0.20 avg=0.21
[11942 | 45034.97] loss=0.21 avg=0.21
[11943 | 45039.36] loss=0.21 avg=0.21
[11944 | 45043.75] loss=0.17 avg=0.21
[11945 | 45048.10] loss=0.18 avg=0.21
[11946 | 45052.44] loss=0.18 avg=0.21
[11947 | 45056.82] loss=0.19 avg=0.21
[11948 | 45061.22] loss=0.15 avg=0.21
[11949 | 45065.60] loss=0.19 avg=0.21
[11950 | 45070.00] loss=0.21 avg=0.21
[11951 | 45074.39] loss=0.18 avg=0.21
[11952 | 45078.80] loss=0.20 avg=0.21
[11953 | 45083.17] loss=0.19 avg=0.21
[11954 | 45087.59] loss=0.19 avg=0.21
[11955 | 45091.96] loss=0.20 avg=0.21
[11956 | 45096.36] loss=0.19 avg=0.21
[11957 | 45100.76] loss=0.21 avg=0.21
[11958 | 45105.15] loss=0.17 avg=0.21
[11959 | 45109.52] loss=0.24 avg=0.21
[11960 | 45113.91] loss=0.18 avg=0.21
[11961 | 45118.30] loss=0.21 avg=0.21
[11962 | 45122.71] loss=0.20 avg=0.21
[11963 | 45127.10] loss=0.21 avg=0.21
[11964 | 45131.49] loss=0.21 avg=0.21
[11965 | 451

In [8]:
# expansion formation rules: seed phrase deleted, truncation at first end 
# of sentence, the last comma, or allow to run off unclipped otherwise
# 
def cleaned(expansions, seed):

   seedbag = word_tokenize(seed)
   cutseed = dt.detokenize(seedbag[1:])
   symlen = len(cutseed)
   cleaned = []
   expansion = ""
   for exp in expansions:
      pos = exp.find(cutseed)
      if pos > 0:
         expansion = exp[pos+symlen:].strip()
      else:
         expansion = exp
      pos = expansion.find(".")
      if pos > 0:
         expansion = expansion[:pos]+"."
      else:
         pos = expansion[::-1].find(",")
         if pos > 0:
            expansion = expansion[:-pos]
      cleaned.append(expansion)
      
   return cleaned

In [9]:
# @every kernel restart
def generateLexeme(sent):

      if sent == "":
         return

      words = word_tokenize(sent)

      lexeme = ""
      count = 0

      if len(words) <= maxwords:
         processLexeme(sent)

      else:
         for word in words:
            count += 1
            if (word in [";", ","] and count > minwords) or count > maxwords:
               if (len(words)-count) >  minwords:
                  processLexeme(dt.detokenize(words[:count]))
                  sentence = dt.detokenize(words[count:])
                  generateLexeme(sentence)
               else:
                  processLexeme(dt.detokenize(words))

               break

In [16]:
# @every kernel restart
def processLexeme(sent):

   # always start from the first phrase
   if not bestLexemes:
      bestLexemes.append(sent)
      print("Leading output with seed phrase: {}".format(sent))
      return
    
   seed = dt.detokenize(bestLexemes[-seedLexemes:])
   print("seed: \"{}\"".format(seed))
   print("\t  processing lexeme: \"{}\"".format(sent))
   
   expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, return_as_list=True, include_prefix=False)
   #expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=".", return_as_list=True, include_prefix=False)
   #expansions += gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=",", return_as_list=True, include_prefix=False)
   expansions = cleaned(expansions, seed)
   #print("  expansion set: {}".format(expansions))
  
   messages = [sent]+expansions
   bestLexeme = sent
  
   gc.collect()

   with tf.Session(config=config) as session:
      session.run(tf.global_variables_initializer())
      session.run(tf.tables_initializer())
      message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})

      corr = np.inner(message_embeddings_, message_embeddings_)
      embeddings = corr[0,1:]
      #print("embeddings: {}".format(embeddings))
      
      bestIndex = np.argmax(embeddings)
      
      if embeddings[bestIndex] >  minsimilarity:
        
         bestLexeme = expansions[bestIndex]
         acceptedLexemes["accepted"] = acceptedLexemes["accepted"]+1
         # join expansion with future lexeme smoothly
        
         originalEnding = sent[-1]
         end = originalEnding if originalEnding in [".",",", ";", "-", "--", "?", "!", "..."] else ""
        
         thisEnding = bestLexeme[-1]
         
         # replace punctuation at end of lexeme if original punctuation existed
         # if new sentence not ending yet, ignore the original markers
         if thisEnding in [".", ",", ";", "?", "!"]:
            bestLexeme = bestLexeme[:-1]+end
      else:
         acceptedLexemes["rejected"] = acceptedLexemes["rejected"]+1  
        
   print("  expanding with: \"{}\" bestscore: {} bestphrase: {}".format(bestLexeme, embeddings[bestIndex], expansions[bestIndex]))
   bestLexemes.append(bestLexeme)
    
   output = dt.detokenize(bestLexemes)
   Path(outputfile).write_text(output)

In [12]:
# !!!!! playground cell 1: test expansion quality of the model

seed = "I have to say that we suck. Moreover, this opportunity leads us to nowhere."

expansions = gpt2.generate(sess, nsamples=10, batch_size=10, length=20, prefix=seed, return_as_list=True, include_prefix=False)

# get around a bug 
print(expansions)
expansions = cleaned(expansions,seed)
print(expansions)

['I have to say that we suck. Moreover, this opportunity leads us to nowhere. I would not say anything to you if you did not give up your thread-and-slope', 'I have to say that we suck. Moreover, this opportunity leads us to nowhere. I wish, instead, to use the example of a poor family in whose case I fail to see', 'I have to say that we suck. Moreover, this opportunity leads us to nowhere.”\n\n“Wait a sec. I will conduct another interview, but I must warn', "I have to say that we suck. Moreover, this opportunity leads us to nowhere. You see, I can't go on like this for numberless reasons. So I take this opportunity", 'I have to say that we suck. Moreover, this opportunity leads us to nowhere. We are sorry that you are not in our hall. Well? Well?”\n\n�', 'I have to say that we suck. Moreover, this opportunity leads us to nowhere.”\n\nFrom somewhere in the fortress a dreadful dull came the friendly peal of the iron', 'I have to say that we suck. Moreover, this opportunity leads us to n

In [13]:
# !!!!! playground cell 2: test lexeme processor

# first lexeme always identity transform
# generateLexeme(" there never was a better place than")

bestLexemes = []
generateLexeme(" there never was a better place than")
print(bestLexemes)

Leading output with seed phrase:  there never was a better place than
[' there never was a better place than']


In [ ]:
# launch RPS from here.
# 
bestLexemes = []
acceptedLexemes = {"accepted":0, "rejected":0}

# large number of samples exhausts Collab memory limit.
nsamples = 10000

# 0.5 is vague, 0.7 reasonably along the source, 0.8 close and 0.9 is literal
minsimilarity = 0.68

sentences = tokenizer.tokenize(input)

for sent in sentences:
   print("Input sentence: {}".format(sent))
   generateLexeme(sent)

output = dt.detokenize(bestLexemes)
#Path(outputfile).write_text(output)
Path(statsfile).write_text(str(acceptedLexemes))

print(output)
print("------------------------------------------------------------")
print(input)


Input sentence: She sat at the window of the train, her head thrown back, one leg stretched across to the empty seat
before her.
Leading output with seed phrase: She sat at the window of the train,
seed: "She sat at the window of the train,"
	  processing lexeme: "her head thrown back,"
  expanding with: "brooding, shaking her head," bestscore: 0.7481966614723206 bestphrase: brooding, shaking her head.
seed: "She sat at the window of the train, brooding, shaking her head,"
	  processing lexeme: "one leg stretched across to the empty seat before her."
  expanding with: "shaking herself, and could not quite make one's feet rest on the floor: another sat there." bestscore: 0.7312697172164917 bestphrase: shaking herself, and could not quite make one's feet rest on the floor: another sat there,
Input sentence: The window frame trembled with the speed of the motion, the pane hung over empty
darkness, and dots of light slashed across the glass as luminous streaks, once in a while.
seed: "She 

In [ ]:
print(bestLexemes)